In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q lime

     |████████████████████████████████| 275 kB 5.4 MB/s 


In [ ]:
import pickle
import pandas as pd
import numpy as np
from keras.models import load_model
from lime.lime_text import LimeTextExplainer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
model_cnn = load_model('/content/drive/MyDrive/MasterThesis/CNN/cnn.h5')

In [ ]:
test_x =  pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/X_test_all.csv')
train_x =  pd.read_csv('/content/drive/MyDrive/MasterThesis/Dataset/X_train_all.csv')
test_sentences = test_x.clean_review.astype(str)
train_sentences = train_x.clean_review.astype(str)

In [ ]:
max_features = 90166
max_length = 50
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_sentences)

In [ ]:
explainer = LimeTextExplainer(class_names=["Negative", "Positive"])

In [ ]:
#As the test_data is already an array of embeddings we can wrap the predict function into a function that takes in a list of strings and returns prediction probabilities
def new_predict(texts):
    _seq = tokenizer.texts_to_sequences(texts)
    _text_data = pad_sequences(
        _seq, maxlen=max_length, padding="post", truncating="post"
    )
    return np.array([[float(1 - x), float(x)] for x in model_cnn.predict(_text_data)])

In [ ]:
idx = 2047   # {845, 1556, 1750, 2150, 2514, 2631, 2652, 2845, 3354, 3447, 3929}
exp = explainer.explain_instance(test_sentences.iloc[idx], new_predict, num_features=50)
#exp.show_in_notebook()

In [ ]:
exp_list = exp.as_list()
sorted(exp_list, key=lambda x: x[1])

**Sample calculation**

Calculating and saving feature importance score for the reviews used as a demonstration example to measure the proposed criteria.

In [ ]:
pos_sample = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/positive_sample_50', 'rb'))
neg_sample = pickle.load(open('/content/drive/MyDrive/MasterThesis/Dataset/negative_sample_50', 'rb')) 

In [ ]:
def lime_calc(sample):
  lime_calc = []
  for i in sample:
    exp = explainer.explain_instance(test_sentences.iloc[i], new_predict, num_features=50) 
    exp_list = exp.as_list()
    sorted_list = sorted(exp_list, key=lambda x: x[1])
    lime_calc.append(sorted_list)
  return lime_calc

In [ ]:
r = lime_calc(pos_sample)
with open('/content/drive/MyDrive/MasterThesis/Dataset/lime_cnn_positive_sample_50', 'wb') as f:
    pickle.dump(r, f) 

r = lime_calc(neg_sample)
with open('/content/drive/MyDrive/MasterThesis/Dataset/lime_cnn_negative_sample_50', 'wb') as f:
    pickle.dump(r, f)        